In [5]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [7]:
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "SousRegimeExacte"

print(COUNTRY_NAME)

MLI Mali


In [8]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


In [9]:
from llm_setup import *

# Get legal documents from existing paperless Postgres

In [10]:
df, documents = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### Get the definitions of the Sous Regimes Standards from Postgres

In [11]:
df_sousregimes_standards = get_postgress_data(sql_files['get_sous_regimes_standards'], db='Ferdi')
df_sousregimes_standards['SousRégimeStandardCode'] = df_sousregimes_standards['SousRégimeStandardCode'].apply(lambda x: x[1:])
df_sousregimes_standards['SousRégimeCode'] = df_sousregimes_standards['SousRégimeCode'].apply(lambda x: x[1:])

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Set the embeddings model and create a vectorized DB for the extracted texts

In [8]:
embed_model = get_voyage_ai_embedding_model()

In [9]:
vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embed_model, chunk_size = int(1024))

loading from local


In [10]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'img_Loi n°2010-060 (30.12.2010) Loi de finances 2011 (Droit-Afrique)'

# Set the llm
the llm will be used to augment the retrieved data from the vector_index

In [15]:
llm = get_open_ai_llm()

# RAG to extract the Sous Regime Exacte
 - create a vector engine using the above created vector database and the title of the document
 - using the llm to augment the retrieved response, and generate the response according to the prompt
 - will first extract the Texte Fiscaux Standard then use this to select the sub category

In [16]:
definitions = df_sousregimes_standards.set_index('SousRégimeStandardCode')['SousRégimeStandardComplet'].to_dict()

In [17]:
from difflib import get_close_matches

def find_most_similar_string(target, string_list):
    # Using difflib's get_close_matches function to find the closest match
    # It returns a list of close matches; we take the first (most similar) one
    closest_matches = get_close_matches(target, string_list, n=1, cutoff=0.0)
    return closest_matches[0] if closest_matches else None

def get_regimes_standards(vector_index, doc_name, llm, definitions=definitions):
    vector_query_engine = create_vector_engine(vector_index, doc_name, llm)

    """
    You are a helpful assistant, with legal document analysis expertise. Please analyze the given context.
    """
    response = vector_query_engine.query(
        f"""
        You are a helpful assistant who can read and analyze legal documents.
        You are able to detect mining sub-regimes, i.e. all mineral rights and authorizations, on which taxation will then depend. 
        It is therefore necessary to review the mining act and extract the exhaustive list of mineral rights and authorizations
        Consider the contents of the document with this title '{doc_name}'. Then, these definitions: {str(definitions)} 
        In which of these categories : {', '.join(list(definitions.keys()))}, is it part of ? Answer with a list of valid categories, keep the categories only. 
        Do not write an introduction, summary, rewrite or anything after the category.
        Do not include any "=" in the final answer. 
        If the document does not match any category, then reply with 'None'.
        If the result is none of the above categories, find the most suitable one, using the definitions and the context.
        """
    )

    print(doc_name, len(response.source_nodes))
    print([node.score for node in response.source_nodes])
    print([node.text for node in response.source_nodes])

    return response.response

### Test the above function

In [18]:
response = get_regimes_standards(
    vector_index=vector_index,
    doc_name = doc_titles.to_list()[-4],
    llm=llm)

print(response)

img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines) 2
[0.86328125, 0.8203125]
["au Mali. Toute personne physique ou morale peut les exploiter, dès lors qu’elle est propriétaire du sol où ils se trouvent ou bien qu’elle en a reçu l’autorisation du propriétaire.\n\nS’il s’agit du domaine de l’Etat, une autorisation d’exploration pourrait être accordée sur demande et postulant et dans les conditions prévues aux articles 22 à 25 du present code minier.\n\nL’exploitation industrielle des gîtes de substances minérales soumis au régime des sols sont soumises aux dispositions du present Code Minier et de ses textes d’application.\n\nARTICLE 11 : Un arrêté conjoint des ministres des Mines et des Finances peut décider le passage à une date déterminée de substances dont les gîtes sont soumis au régime des carrières vers le régime des mines.\n\nDe même, un arrêté conjoint des ministres chargés des Mines et des Finances peut décider le passage, à une date déterminée, de substance

In [20]:
sousregimes_standards = list()

for doc in doc_titles:
    try:
        categ = get_regimes_standards(vector_index=vector_index, doc_name = doc, llm = llm)
    except Exception as e:
        print(e)

        categ = "non_determined"

    sousregimes_standards.append((doc, categ))

img_Loi n°2010-060 (30.12.2010) Loi de finances 2011 (Droit-Afrique) 2
[0.58984375, 0.5234375]
["<tr>\n    <td></td>\n    <td>1 235 480</td>\n    <td></td>\n    <td>73 099</td>\n    <td></td>\n    <td>1 308 579</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td>1 235 480</td>\n    <td></td>\n    <td>73 099</td>\n    <td></td>\n    <td>1 308 579</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td>0</td>\n    <td></td>\n    <td>50 000</td>\n    <td></td>\n    <td>50 000</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td></td>\n    <td></td>\n    <td>50 000</td>\n    <td></td>\n    <td>50 000</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td>0</td>\n    <td></td>\n    <td>260 000</td>\n    <td>-40 000</td>\n    <td>220 000</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td></td>\n    <td></td>\n    <td>6 871 366</td>\n    <td>-923 927</td>\n    <td>5 947 439</td>\n  </tr>\n  <tr>\n    <td></td>\n    <td></td>\n    <td></td>\n    <td>2 115 000</td>\n    <td>-46 410</td>\n    <td>2 068 590</td>\n  </tr>\n  <tr>\n  

In [21]:
sousregimes_standards

[('img_Loi n°2010-060 (30.12.2010) Loi de finances 2011 (Droit-Afrique)',
  'None'),
 ('Loi n°2001-075 (18.07.2001) Code des douanes 2001 (eRegulations Mali)',
  'Empty Response'),
 ('img_Loi n°2011-078 (23.12.2011) Annexe fiscale Loi de finances 2012 (Investir-Afrique)',
  'None'),
 ('Ordonnance n°2020-013 (21.12.2020) Loi de finances 2021 (Ministère des Finances)',
  'None'),
 ('img_Loi n°2008-021 (22.07.2008) Modification LPF 2008 (Droit-Afrique)',
  'None'),
 ('Loi n°2012-063 (26.12.2012) Annexe fiscale Loi de finances 2013 (DGI)',
  'None'),
 ('Tarif douanier CEDEAO 2017 amendé 2020 (DGD)', 'None'),
 ('Loi n°2016-056 (21.12.2016) Annexe fiscale Loi de finances 2017 (Ministère des Finances)',
  'None'),
 ('Ordonnance n°2020-001 (04.0902020) Loi de finances rectificative 2020 (Ministère des Finances)',
  'Empty Response'),
 ('Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)',
  'None'),
 ('Loi n°2001-064 (09.07.2001) Modification CGI 2001 (SGG)', 'Emp

In [40]:
df_SousRegimesExactes = pd.DataFrame.from_dict(dict(sousregimes_standards), orient='index', columns=[table_column_name])
df_SousRegimesExactes = df_SousRegimesExactes[~df_SousRegimesExactes[table_column_name].isin(["None", "Empty Response"])]

In [44]:
import re

df_SousRegimesExactes.SousRegimeExacte.apply(lambda x: re.split(r'[\n,]+', x)).explode()

img_Loi n°2006-067 (29.12.2006) Code général des impôts 2016 (DGI)             Min_Rech_Prospect
img_Loi n°2006-067 (29.12.2006) Code général des impôts 2016 (DGI)                 Min_Rech_Rech
img_Loi n°2006-067 (29.12.2006) Code général des impôts 2016 (DGI)           Min_Expl_ExplPetite
img_Loi n°2006-067 (29.12.2006) Code général des impôts 2016 (DGI)          Min_Expl_ExplArtisan
Décret n°1999-255 (15.09.1999) Application Code minier 1999 (SGG)                Min_Rech_Explor
                                                                                   ...          
img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines)      Min_Expl_ExplArtisan
img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines)     Min_Expl_ExplSemiMéca
img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines)       Min_Expl_ExplPetite
Loi n°2015-035 (16.07.2015) Code pétrolier 2015 (Droit-Afrique)               Pétrol_Rech_Explor
Loi n°2015-035 (16.07.2015) Co

In [1]:
df_SousRegimesExactes = df_SousRegimesExactes.SousRegimeExacte.apply(lambda x: re.split(r'[\n,]+', x)).explode()

df_SousRegimesExactes.to_csv(f"../output/{table_column_name}_{COUNTRY_NAME}.csv", index=True)

df_SousRegimesExactes

NameError: name 'df_SousRegimesExactes' is not defined

# Periodes

In [2]:
import pandas as pd

In [3]:
df_SousRegimesExactes = pd.read_csv(f"../output/SousRegimeExacte_MLI Mali.csv")

In [12]:
definitions = df_sousregimes_standards.set_index('SousRégimeStandardCode')['SousRégimeStandardComplet'].to_dict()